In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 例外処理用のライブラリをインポート
from selenium.common.exceptions import NoSuchElementException, WebDriverException, TimeoutException
import sys

import pandas as pd
import numpy as np

seleniumマネジャーが利用可能かハージョン確認  
4.6以前であれば更新

In [ ]:
print(selenium.__version__)

## 詳細ページに遷移するための関数
* ~~WebdriverExceptionが発生した場合に待機して、リロードを行う~~<br>
* ~~2間隔でページ全体が更新されたか、最大3秒間確認する~~
* 3秒以上の動画広告に対処するため、この関数ではページ遷移と、エラー発生時のリロードのみ行う。<br>
5回リロードしても同じエラーが発生する場合には強制終了を行う

In [ ]:
def Safe_navigation(href, i=0):
  try:
    driver.get(href)
    # 例外オブジェクトを e としてキャッチ
  except Exception as e:
    if i >= 5:
      try:
        sys.exit()
      except SystemExit:
        print('ページ遷移不可のため、強制終了')
      raise
    else:
        print(f"Navigation error: {e}")
        i += 1
        Safe_navigation(href, i)

## 詳細ページからデータを取得する関数

In [ ]:
#絞込に従う、ランキングtop4~5店の詳細情報を取得する
def store_detail(name_list, points_list, href_list, df_pref, pref_name):
  # top4~50の店から店舗詳細情報を取得
  for i in range(len(href_list)):
    Safe_navigation(href_list[i])
    # 住所が登録されている要素を取得
    address = Get_address()
    # 緯度経度の要素を取得する
    latitude,  longitude = Get_map()
    # 駐車場の要素を取得。（classの登録がなかったため、テキスト部分で条件指定
    parking = Get_parking()
    # 定休日の要素を取得。（classの登録がなかったため、テキスト部分で条件指定）
    holiday = Get_holiday()
    Mk_detail_list(name_list[i], points_list[i], pref_name, address, latitude, longitude, parking, holiday, df_pref)
  # 定期的なキャッシュの削除
  driver.get('chrome://settings/clearBrowserData')
  # ランキングのページに戻り、都道府県のプルダウンを取得できるようにする
  driver.get('https://ramendb.supleks.jp/rank')
  # ページ全体がロードされるのを最大3秒間待機する
  WebDriverWait(driver, 3).until(
      lambda d: d.execute_script('return document.readyState') == 'complete'
  )
    
def Get_address():
  try:
    #addressの要素を2秒おきに確認しながら、最大3秒待機する
    address_ele = WebDriverWait(driver, 3, poll_frequency=2).until(
      EC.presence_of_element_located((By.XPATH, '//tbody/tr/td/span[@itemprop="address"]'))
    )
    address = address_ele.text
  # 住所が取得できなければ、Noneを取得
  except Exception as e:
    print(e)
    address = None
  return address

def Get_map():
  try:
    # mapの要素を確認しながら、最大3秒待機する。addressと同じページ内なので、確認秒数はデフォルトの500ミリ秒とする
    map_data_ele = WebDriverWait(driver, 3).until(
      EC.presence_of_element_located((By.ID, 'gpetitembedmap'))
    )
    map_data = map_data_ele.get_attribute('src')
    # srcの要素値を緯度経度だけ切り離す
    # q= という文字が入るので、スライスで取り除く
    map_data = map_data.split('&')[1][2:]
    # 緯度経度に分割する
    map_data = map_data.split(',')
    latitude = map_data[0]
    longitude = map_data[1]
  # 緯度経度が取得できなければ、Noneを取得
  except Exception as e:
    print(e)
    latitude = None
    longitude = None
  return latitude, longitude
        
def Get_parking():
  try:
    th_element = WebDriverWait(driver, 3).until(
      EC.presence_of_element_located((By.XPATH, '//tbody/tr/th[contains(text(), "駐車場")]'))
    )
    parking = th_element.find_element(By.XPATH, "./following-sibling::td").text
  except Exception as e:
    print(e)
    parking = None
  return parking
        
def Get_holiday():
  try:
    th_element = WebDriverWait(driver, 3).until(
      EC.presence_of_element_located((By.XPATH, '//tbody/tr/th[contains(text(), "定休日")]'))
    )
    holiday = th_element.find_element(By.XPATH, "./following-sibling::td").text
  except Exception as e:
    print(e)
    holiday = None
  return holiday

# 店舗名とポイント、店舗詳細を店舗ごとにリスト化する
def Mk_detail_list(name, points, pref_name, address, latitude, longitude, parking, holiday, df_pref):
  detail_list = [name]
  detail_list.append(points)
  detail_list.append(pref_name)
  detail_list.append(address)
  detail_list.append(latitude)
  detail_list.append(longitude)
  detail_list.append(parking)
  detail_list.append(holiday)
  print(detail_list)
  df_pref.loc[len(df_pref)] = detail_list

## 都道府県ごとのランキングを取得する関数

In [ ]:
# 都道府県ごとのランキングデータフレームを作成する
def Pref_cycle(df_pref):
  for i in range(1, 48):
    # 都道府県ごとのデータ取得
    pref_element = driver.find_element(By.XPATH, '//div[@class="wrap"]/nav/div[@class="left"]/select[@name="state"]')
    # 都道府県のselect要素をselectオブジェクトへ変換
    pref_select = Select(pref_element)
    # オブジェクトからkeyを取得する
    pref_text_list = [pref.text for pref in pref_select.options]
    # オブジェクトからvalue属性のリストを作成する
    pref_val_list = [pref.get_attribute('value') for pref in pref_select.options]
    # 都道府県の要素をドロップダウンから選択して
    pref_select.select_by_value(pref_val_list[i])
    # 表示ボタンの要素を取得できるまで最大3秒間待機し、2秒ごとに要素が表示されるか確認
    viewer = WebDriverWait(driver, 3, poll_frequency=2).until(
        EC.element_to_be_clickable((By.XPATH, '//div[@class="left"]/input[@class="formbtn"]'))
    )
    # 要素（表示ボタン）が表示されるまでスクロールする
    driver.execute_script("arguments[0].scrollIntoView();", viewer)
    viewer.click()

    # 必要な要素(top3の店名とポイント, top4~50の店名とポイント)が表示されるまで最大5秒間待機
    store_ele_top3 = WebDriverWait(driver, 5, poll_frequency=2).until(
      EC.presence_of_all_elements_located((By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="name"]/span/a'))
    )
    store_top3_point = WebDriverWait(driver, 5).until(
      EC.presence_of_all_elements_located((By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="point"]'))
    )
    store_ele = WebDriverWait(driver, 5).until(
      EC.presence_of_all_elements_located((By.XPATH, '//table[@class="rank"]/tbody/tr/td/span[@class="name"]/a'))
    )
    store_point = WebDriverWait(driver, 5).until(
      EC.presence_of_all_elements_located((By.XPATH, '//table[@class="rank"]/tbody/tr/td[@class="point"]'))
    )
    # 取得したtop3の店名とポイント、詳細ページのurlをリスト形式で保存
    store_name_top3_list = [store_top3.text for store_top3 in store_ele_top3]
    store_point_top3_list = [store_top3.text for store_top3 in store_top3_point]
    href_list_top3 = [ hrefs.get_attribute('href') for hrefs in store_ele_top3]
    
    # 取得したtop4~50の店名とポイント、詳細ページのurlをリスト形式で保存
    store_name_list = [store.text for store in store_ele]
    store_point_list = [store.text for store in store_point]
    href_list = [hrefs.get_attribute('href') for hrefs in store_ele]

    # リストをそれぞれ結合する
    store_name_list = store_name_top3_list + store_name_list
    store_point_list = store_point_top3_list + store_point_list
    href_list = href_list_top3 + href_list

    store_detail(store_name_list, store_point_list, href_list, df_pref, pref_text_list[i])

## 都道府県ごとのランキングを取得したい場合に実行する

In [ ]:
# 必要に応じてoptionsを設定
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# 暗黙的な待機
driver.implicitly_wait(3)
driver.get('https://ramendb.supleks.jp/rank')

# # データフレームの枠組みを作成する
columns_list_detail = ['name', 'point', 'pref', 'address', 'latitude', 'longitude', 'parking', 'holiday']
df_pref = pd.DataFrame(columns = columns_list_detail)

Pref_cycle(df_pref)
df_pref.to_csv('../data/pref_rank.csv',index=False)